#### 先找出23-24營業毛利資料的共同公司

In [ ]:
import pandas as pd
sc_24=pd.read_excel("SC_上市櫃(23-24).xlsx")

In [5]:
import pandas as pd
import numpy as np
sc_24=pd.read_excel("SC_上市櫃(23-24).xlsx",sheet_name="Sheet1")
sc_24_overall=pd.read_excel("SC_上市櫃(23-24).xlsx",sheet_name="大盤指數_24")
sc_23=pd.read_excel("SC_上市櫃(23).xlsx",sheet_name="Sheet1")
sc_23_overall=pd.read_excel("SC_上市櫃(23).xlsx",sheet_name="大盤指數_23")
sc_24.loc[:,"代號"]=sc_24.loc[:,"代號"].astype(int)
sc_23.loc[:,"代號"]=sc_23.loc[:,"代號"].astype(int)
sc_24=sc_24.sort_values(by="代號",ascending=True)
sc_23=sc_23.sort_values(by="代號",ascending=True)

with pd.ExcelWriter('SC_上市櫃(23-24).xlsx', engine='openpyxl') as writer:
    sc_24.to_excel(writer, index=False,sheet_name='Sheet1')
    sc_24_overall.to_excel(writer, index=False,sheet_name='大盤指數_24')

with pd.ExcelWriter('SC_上市櫃(23).xlsx', engine='openpyxl') as writer:
    sc_23.to_excel(writer, index=False,sheet_name='Sheet1')
    sc_23_overall.to_excel(writer, index=False,sheet_name='大盤指數_23')




In [19]:
sc_24=pd.read_excel("SC_上市櫃(23-24).xlsx",sheet_name="Sheet1")
sc_24_overall=pd.read_excel("SC_上市櫃(23-24).xlsx",sheet_name="大盤指數_24")
sc_23=pd.read_excel("SC_上市櫃(23).xlsx",sheet_name="Sheet1")
sc_23_overall=pd.read_excel("SC_上市櫃(23).xlsx",sheet_name="大盤指數_23")

In [6]:
same_2324,arr_24,arr_23=np.intersect1d(sc_24["代號"].values.flatten(),sc_23["代號"].values.flatten(),return_indices=True)

print(len(sc_24.loc[arr_24,"代號"]))
print(len(sc_23.loc[arr_23,"代號"]))
with pd.ExcelWriter('SC_上市櫃(23-24).xlsx', engine='openpyxl') as writer:
    sc_24.iloc[arr_24,:].to_excel(writer, index=False,sheet_name='Sheet1')
    sc_24_overall.to_excel(writer, index=False,sheet_name='大盤指數_24')

with pd.ExcelWriter('SC_上市櫃(23).xlsx', engine='openpyxl') as writer:
    sc_23.iloc[arr_23,:].to_excel(writer, index=False,sheet_name='Sheet1')
    sc_23_overall.to_excel(writer, index=False,sheet_name='大盤指數_23')





1929
1929


### 再找出23_24資料與現有上市櫃公司的共同公司

#### <span style = "color : #FA5C5C">發現 : 金融業無營業毛利 </span>
#### <a href="https://ppt.cc/fZ35cx"><img src="https://ppt.cc/fZ35cx@.png" border="0" alt="PPT.cc縮圖服務" title="PPT.cc縮圖服務"></a>


In [44]:
df_fs=pd.read_excel("上市櫃公司.xlsx")
sc_24["營業毛利_23"]=sc_23["營業毛利"]
sc_24=sc_24.fillna(0)
with pd.ExcelWriter('SC_上市櫃(23-24).xlsx', engine='openpyxl') as writer:
    sc_24.to_excel(writer, index=False,sheet_name='Sheet1')
    sc_24_overall.to_excel(writer, index=False,sheet_name='大盤指數_24')


same_2324,arr_24,df_fs_index=np.intersect1d(sc_24["代號"].values.flatten(),df_fs["代號"].values.flatten(),return_indices=True)

print(len(sc_24.loc[arr_24,"代號"]))
with pd.ExcelWriter('SC_上市櫃(23-24).xlsx', engine='openpyxl') as writer:
    sc_24.iloc[arr_24,:].to_excel(writer, index=False,sheet_name='Sheet1')
    sc_24_overall.to_excel(writer, index=False,sheet_name='大盤指數_24')



1915


### 將上市櫃公司資料的TSE產業名稱納入 sc_24中

### bug : 不知為何比對下會少 中化控股 ( 遺漏公司 : 1家)

###  <span style="color: red">原因 : 23年資料無此家公司，在執行上忽略了要以23年資料比對25年資料 </span>

#### <span style="color: green"> 解決方式 : 先另外儲存共同企業的產業名稱，再用pd.concat(axis=1)合併2個excel </span>


#### axis=1 : 左右拼接   axis = 0 ，上下拼接

#### <span style="color : #FF9B51">另外先儲存indus_name資料，在資料合併上更有效 </span>

In [ ]:
indus_name=df_fs.loc[df_fs_index,"TSE產業名稱"]
print(indus_name)
sc_24=pd.concat([sc_24,indus_name],axis=1)

with pd.ExcelWriter('SC_上市櫃(23-24).xlsx', engine='openpyxl') as writer:
    sc_24.to_excel(writer, index=False,sheet_name='Sheet1')
    sc_24_overall.to_excel(writer, index=False,sheet_name='大盤指數_24')

0       水泥工業
1       水泥工業
2       水泥工業
3       水泥工業
4       水泥工業
        ... 
1911    綠能環保
1912    鋼鐵工業
1913    運動休閒
1914    鋼鐵工業
1915    存託憑證
Name: TSE產業名稱, Length: 1915, dtype: object


NameError: name 'sc_24_combine' is not defined

### <span style="color:white"> SC 變數計算: </span>
### step 1 : 先用groupby將資料分組，算出各產業的產業平均毛利
### step 2 : 再用series.map() 將對應的產業平均毛利，對應到sc_24中產業名稱後的儲存格
### step 3 : sc = 單一企業毛利成長率 / 對應產業毛利成長率

In [ ]:
sc_24=pd.read_excel("SC_上市櫃(23-24).xlsx",sheet_name="Sheet1")
margin_avg_indus_24=sc_24.groupby(["TSE產業名稱"])["營業毛利_24"].mean()
margin_avg_indus_23=sc_24.groupby(["TSE產業名稱"])["營業毛利_23"].mean()
print("2024年各產業平均毛利:")
print(margin_avg_indus_24)

<class 'pandas.core.series.Series'>
2024年各產業平均毛利:
TSE產業名稱
光電業         1.585057e+06
其他          2.153075e+06
其他電子業       6.415856e+06
化學工業        1.977386e+06
半導體業        1.345211e+07
塑膠工業        2.200724e+06
存託憑證        2.527030e+06
居家生活        3.091832e+06
建材營造        1.846444e+06
數位雲端        1.224164e+06
文化創意業       1.406910e+06
橡膠工業        4.121545e+06
水泥工業        6.901510e+06
汽車工業        4.155878e+06
油電燃氣業       1.824572e+06
玻璃陶瓷        1.448898e+06
生技醫療業       9.549394e+05
紡織纖維        2.107969e+06
綠能環保        8.004959e+05
航運業         1.443690e+07
觀光餐旅        1.372798e+06
貿易百貨        7.389943e+06
資訊服務業       8.990895e+05
農業科技        2.249462e+05
通訊網路業       3.059771e+06
造紙工業        3.831807e+06
運動休閒        6.644106e+06
金融業         1.480470e+06
鋼鐵工業        1.862419e+06
電器電纜        1.845480e+06
電子工業        4.847270e+05
電子通路業       4.078406e+06
電子零組件       2.697130e+06
電機機械        1.321756e+06
電腦及週邊設備業    7.419162e+06
食品工業        8.662661e+06
Name: 營業毛利_24, dtype: float64


In [71]:
sc_24["產業平均毛利_24"]=sc_24["TSE產業名稱"].map(margin_avg_indus_24)
sc_24["產業平均毛利_23"]=sc_24["TSE產業名稱"].map(margin_avg_indus_23)
sc_24["產業銷貨毛利增長率"]=(sc_24["產業平均毛利_24"]-sc_24["產業平均毛利_23"])/sc_24["產業平均毛利_23"]
sc_24["營業毛利成長率_2324"]=sc_24["營業毛利成長率_2324"]/100
sc_24["SC變數"]=sc_24["營業毛利成長率_2324"]/sc_24["產業銷貨毛利增長率"]
with pd.ExcelWriter('SC_上市櫃(23-24).xlsx', engine='openpyxl') as writer:
    sc_24.to_excel(writer, index=False,sheet_name='Sheet1')
    sc_24_overall.to_excel(writer, index=False,sheet_name='大盤指數_24')
